# ZBIW-Zertifikatskurs "Data Librarian"
# Modul 2.1: Strukturierte Daten und Metadaten

## Text-Dateien

Recap zu Lesen und Schreiben von Textdateien in Python:

In [ ]:
# Datei im 'read-only' Modus öffnen
file_for_reading = open('data/lorem_ipsum.txt', 'r')

In [ ]:
# nach getaner Arbeit muss die Datei wieder geschlossen werden:
file_for_reading.close()

In [ ]:
# besser: 'with' Block benutzen, damit Dateien automatisch geschlossen werden
with open('data/lorem_ipsum.txt', 'r') as infile:
    data = infile.readlines()

`data` ist nun ein Listenobjekt, wobei jedes Listenelement eine Textzeile aus der Datei enthält:

In [ ]:
type(data)

In [ ]:
data

Das zeilenweise Einlesen ist ebenfalls möglich:

In [ ]:
with open('data/lorem_ipsum.txt', 'r') as infile:
    for line in infile:
        print(line.strip()) # entferne überflüssigen Whitespace

Man kann den Code noch etwas verkürzen, wenn man die Referenz auf `infile` nicht zusätzlich benötigt:

In [ ]:
for line in open('data/lorem_ipsum.txt', 'r'):
    print(line.strip())

Das Schreiben von Dateien funktioniert analog zum Lesen - hier wird der Modus-Parameter der `open`-Funktion auf `w` (write) gesetzt:

In [ ]:
# Datei im 'write' Modus öffnen - bestehende Datei wird überschrieben!
with open('output/writing_file.txt', 'w') as outfile:
    outfile.writelines(data[0])

Alternativ sorgt der Parameterwert `a` (append) dafür, dass bestehender Inhalt in der Ausgabedatei nicht überschrieben wird. Stattdessen wird der neue Inhalt angehangen:

In [ ]:
# Datei im 'append' Modus öffnen - Geschriebenes wird an bestehenden Inhalt angehangen
with open('output/appending_file.txt', 'a') as outfile:
    outfile.writelines(data[0])

---

## CSV

Wie wir gesehen haben, ist das CSV-Format ein textbasiertes Datenformat. Daher kann es wie eine reguläre Textdatei in Python behandelt werden. Dies hat allerdings seine Tücken, wie das folgende Beispiel zeigt.

### Beispiel: NYT-Beststellerliste

Als erstes sehen wir uns eine CSV-Datei ohne Header an. Hier ist der Inhalt der ersten 10 Zeilen:

In [ ]:
!head data/nyt_bestsellers.csv

Wir sehen, dass jeder Datensatz aus 3 Datenfeldern besteht, die wir als Rang, Titel und Genre identifizieren. Entsprechend versuchen wir, diese Werte getrennt auszulesen:

In [ ]:
# Beispiel: CSV-Datei auch als Text behandeln
for line in open('data/nyt_bestsellers.csv', 'r'):
    row = line.split(',') # problematisch, da Komma auch Teil der Daten sein könnte
    rank = row[0]  # Zuordnung der Felder
    title = row[1]
    genre = row[2].strip()
    print("Rank: {}\nTitle: {}\nGenre: {}\n".format(rank, title, genre))

### Beispiel: Bibsonomy-Export

Vorbereitung: Auf [bibsonomy](https://www.bibsonomy.org) die Suchergebnisse (z.B. ["Bibliothek"](https://www.bibsonomy.org/export/search/Bibliothek)) als CSV exportieren (Limit vorher auf 1000 erhöhen), lokal speichern.

Diese Datei enthält einen Header, der uns die Keys zu den Werten in den Datensätzen liefert:

In [ ]:
!head -n 1 data/search_Bibliothek.csv

In [ ]:
path = 'data/search_Bibliothek.csv'

### Naiver Ansatz über Einlesen als Text und splitten am Komma:

In [ ]:
with open(path) as csvfile:
    lines = csvfile.readlines()

header = lines[0].strip().split(',') # Kopfzeile = erste Datenreihe
print(header)

In [ ]:
first_line = lines[1].strip().split(',') # 2. Datenreihe = 1. Datensatz; splitten der Datenfelder am Trennzeichen

for i in range(len(header)):
    print(header[i], ':', first_line[i]) # Zuordnung der Header-Felder zu den Daten

Wie wir sehen, funktioniert dieser naive Ansatz nicht, da auch die Datenfelder Kommata enthalten und somit unser Trennzeichen nicht eindeutig ist. Die Aufteilung der Daten in die korrekten Felder klappt so nicht.

### Nutzen der CSV Library

Python bietet mit dem csv Modul eine komforable Möglichkeit, CSV Dateien zu lesen und zu schreiben.

In [ ]:
import csv

In [ ]:
with open(path, 'r') as csvfile:
    data = [row for row in csv.reader(csvfile, delimiter=",")]

In [ ]:
header = data[0] # Kopfzeile ist auch hier wieder die erste Datenreihe
print(header)

In [ ]:
first_line = data[1] # zweite Datenreihe = 1. Datensatz
for i in range(len(first_line)):
    print(header[i], ':', first_line[i])

Nun hat die Zuordnung der Daten zu den Header Feldern korrekt geklappt.

Eine weitere praktische Funktion des CSV-Moduls ist die Möglichkeit, die Daten unmittelbar in ein `Dictionary` einzulesen. Dies setzt voraus, dass die Datei einen Header enthält, welcher die Keys für das Dictionary bereitstellt.

In [ ]:
# einlesen mit DictReader
with open(path, 'r') as csvfile:
    reader = csv.DictReader(csvfile, delimiter=',')
    for row in reader:
        print("Title: {}\nAuthors: {}\nISBN: {}\n".format(row['Title'], row['Author'], row['ISBN']))

Im Falle des fehlenden Headers werden die Keys manuell definiert:

In [ ]:
header=['rank', 'title', 'genre']

with open('data/nyt_bestsellers.csv', 'r') as csvfile:
    reader = csv.DictReader(csvfile, delimiter=',', fieldnames=header)
    for row in reader:
        print("Rank: {}\nTitle: {}\nGenre: {}\n".format(row['rank'], row['title'], row['genre']))

---

## JSON

Wir sparen uns jetzt den Versuch, JSON Dateien ebenfalls wie normale Textdateien zu behandeln, da die hierarchische Strukur ungleich schwerer manuell zu parsen wäre.

Stattdessen nutzen wir auch hier das passende Python Modul.

In [ ]:
import json

Als Datensatz verwenden wir wieder [bibsonomy](https://www.bibsonomy.org/export/search/Bibliothek). Wir stellen das Exportformat auf JSON um und stellen fest, dass uns diesmal kein Download angeboten wird, sondern wir auf eine neue URL umgeleitet werden. Diese notieren wir:

In [ ]:
url = "https://www.bibsonomy.org/json/search/Bibliothek?items=1000&duplicates=merged"

Es gibt nun zwei mögliche Wege, diese Daten über Python von der url abzurufen.

Mit der built-in library urllib:

In [ ]:
import urllib.request
import urllib.parse

f = urllib.request.urlopen(url) # f ist eine HTTPResponse
result = f.read().decode('utf-8') # utf-8 ist oft die richtige Wahl

In [ ]:
print(type(result))

In [ ]:
# Ergebnis ist str, muss erst noch als JSON verarbeitet werden (Ergebnis = dict)
data = json.loads(result) # json.loads() lädt JSON Daten aus einem String
print(type(data))

Mit third-party library requests:

In [ ]:
import requests

In [ ]:
result = requests.get(url) # result ist ein requests.models.Response Objekt

In [ ]:
data = result.json() # das Response-Objekt bietet diese nützliche Methode an (Ergebnis = dict)

In [ ]:
type(data)

In [ ]:
print(data)

In [ ]:
# print in a prettier way
print(json.dumps(data, indent=2))

In [ ]:
# same but using pprint (pretty print) module
from pprint import pprint
pprint(data)

Wir können die Daten nun wie ein normales Dictionary behandeln und damit arbeiten:

In [ ]:
print(data.keys()) # Anzeige der vorhandenen Keys

In [ ]:
items = data['items']
print(type(items))
print(len(items))

In [ ]:
publications = [item for item in items if item['type'] == 'Publication']
print(len(publications))

In [ ]:
pubs_with_abstracts = [item for item in publications if 'abstract' in item]
print(len(pubs_with_abstracts))

Wir wollen nun die Publikationen nach Sprache filtern und nur die Deutschen behalten. Zum Detektieren der Sprache benötigen wir ein weiteres Modul:

In [ ]:
from langdetect import detect

In [ ]:
german_pubs = [item for item in publications if detect(item['label'].strip()) == 'de']

In [ ]:
len(german_pubs)

In [ ]:
for item in german_pubs:
    if 'abstract' in item:
        print(item['abstract'])

### JSON-Dateien schreiben

JSON-Objekte können zur Persistierung natürlich auch wieder als Textdatei gespeichert werden. Hierfür steht die Funktion `json.dump` zur Verfügung, welche als Parameter die Daten (Dictionary oder List), ein Datei(-ähnliches) Objekt, sowie optional die Anzahl der Leerzeichen für Einrückung annimmt:

In [ ]:
with open('output/german_publications.json', 'w') as outfile:
    json.dump(german_pubs, outfile, indent=4)

Parallel zu `json.dump` gibt es auch noch `json.dumps`, welche den String zurückgibt wie er in eine Datei geschrieben werden würde, ohne dies tatsächlich zu tun. So kann man ggf. das Ausgabe-Ergebnis vorher überprüfen:

In [ ]:
print(json.dumps(german_pubs, indent=2))